In [1]:
import numpy as np
import sounddevice as sd
import scipy.signal as signal
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

In [3]:
def generate_sweep(fs, duration, f_start, f_end):
    t = np.linspace(0, duration, int(fs * duration), endpoint=False)
    sweep = signal.chirp(t, f0=f_start, f1=f_end, t1=duration, method='logarithmic')
    wav.write("sweep_stimulus.wav", fs, sweep)
    return t, sweep

In [4]:
def measure_system_response(fs, duration, f_start, f_end, input_gain=0.1):
    print("Generating sweep...")
    t, sweep = generate_sweep(fs, duration, f_start, f_end)
    sweep = sweep * input_gain

    print(sd.query_devices())
    print("Playing and recording...")
    recording = sd.playrec(sweep, samplerate=fs, channels=2, dtype='float64')
    sd.wait()
    print("Done recording.")
    return recording[:, 1], recording[:, 0]

In [5]:
fs = 44100
duration = 20
f_start = 5
f_end = 800

input_signal, output_signal = measure_system_response(fs, duration, f_start, f_end)

Generating sweep...
> 0 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 1 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  2 Microsoft Teams Audio, Core Audio (1 in, 1 out)
  3 ZoomAudioDevice, Core Audio (2 in, 2 out)
  4 Teensy & HP, Core Audio (0 in, 0 out)
  5 Aggregate Device, Core Audio (0 in, 0 out)
  6 Teensy and Speakers, Core Audio (0 in, 2 out)
  7 teensy io combined, Core Audio (0 in, 0 out)
Playing and recording...


PortAudioError: Error opening Stream: Invalid number of channels [PaErrorCode -9998]

In [ ]:
scaled_input = np.int16(input_signal / np.max(np.abs(input_signal)) * 32767)
scaled_output = np.int16(output_signal / np.max(np.abs(output_signal)) * 32767)
stereo_recording = np.column_stack((scaled_input, scaled_output))
wav.write("sweep_response.wav", fs, stereo_recording)

In [ ]:
fs, data = wav.read("sweep_response.wav")
input_signal = data[:, 0].astype(np.float64) / 32767
output_signal = data[:, 1].astype(np.float64) / 32767

In [ ]:
def compute_transfer_function_fft(input_signal, output_signal, fs):
    N = len(input_signal)
    window = np.hanning(N)
    X = np.fft.rfft(input_signal * window)
    Y = np.fft.rfft(output_signal * window)
    freqs = np.fft.rfftfreq(N, d=1/fs)
    X = np.where(np.abs(X) < 1e-12, 1e-12, X)
    H = Y / X
    return freqs, H

In [ ]:
freqs, H_measured = compute_transfer_function_fft(input_signal, output_signal, fs)

valid = (freqs >= f_start) & (freqs <= f_end)
freqs = freqs[valid]
H_measured = H_measured[valid]

In [ ]:
def smooth_magnitude(freqs, magnitude, smoothing_octaves=0.1):
    smoothed = np.zeros_like(magnitude)
    for i, f in enumerate(freqs):
        if f <= 0:
            smoothed[i] = magnitude[i]
            continue
        f_low = f * 2**(-smoothing_octaves / 2)
        f_high = f * 2**(smoothing_octaves / 2)
        mask = (freqs >= f_low) & (freqs <= f_high)
        smoothed[i] = np.mean(magnitude[mask])
    return smoothed

In [ ]:
H_mag_smoothed = smooth_magnitude(freqs, np.abs(H_measured), smoothing_octaves=0.1)
H_measured = H_mag_smoothed * np.exp(1j * np.angle(H_measured))

In [ ]:
def estimate_wideband_gain(freqs, H_measured, fit_freq_range=None):
    if fit_freq_range:
        fmin, fmax = fit_freq_range
        mask = (freqs >= fmin) & (freqs <= fmax)
        H_subset = H_measured[mask]
    else:
        H_subset = H_measured
    gain_linear = np.mean(np.abs(H_subset))
    return 20 * np.log10(gain_linear)

In [ ]:
wb_gain_db = estimate_wideband_gain(freqs, H_measured, (10, 40))
print(f"Estimated wideband gain: {wb_gain_db:.2f} dB")
H_measured /= 10**(wb_gain_db / 20)

In [ ]:
def peaking_eq_biquad(f0, Q, gain_db, fs):
    A = 10**(gain_db / 40)
    w0 = 2 * np.pi * f0 / fs
    alpha = np.sin(w0) / (2 * Q)
    b0 = 1 + alpha * A
    b1 = -2 * np.cos(w0)
    b2 = 1 - alpha * A
    a0 = 1 + alpha / A
    a1 = -2 * np.cos(w0)
    a2 = 1 - alpha / A
    b = np.array([b0, b1, b2]) / a0
    a = np.array([1.0, a1 / a0, a2 / a0])
    return b, a

def fit_peaking_eq_q_only(freqs, target_H, fs, fit_freq_range=None):
    if fit_freq_range:
        fmin, fmax = fit_freq_range
        mask = (freqs >= fmin) & (freqs <= fmax)
        freqs_fit = freqs[mask]
        target_H_fit = target_H[mask]
    else:
        freqs_fit = freqs
        target_H_fit = target_H

    mags = np.abs(target_H_fit)
    min_idx = np.argmin(mags)
    min_gain = 20 * np.log10(mags[min_idx])
    f0 = freqs_fit[min_idx]
    gain_db = min_gain

    def q_objective(Q):
        if Q <= 0.05 or Q > 50:
            return 1e6
        b, a = peaking_eq_biquad(f0, Q, gain_db, fs)
        w = 2 * np.pi * freqs_fit / fs
        _, H_fit = signal.freqz(b, a, worN=w)
        error = np.abs(H_fit) - np.abs(target_H_fit)
        return np.sum(error**2)

    result = minimize_scalar(q_objective, bounds=(0.1, 50), method='bounded')
    Q_opt = result.x
    return f0, Q_opt, gain_db

In [ ]:
f0, Q, gain_db = fit_peaking_eq_q_only(freqs, H_measured, fs, (20, 500))
b_fit, a_fit = peaking_eq_biquad(f0, Q, gain_db, fs)
_, H_fit = signal.freqz(b_fit, a_fit, worN=2 * np.pi * freqs / fs)

print(f"Fitted Peaking EQ:\n  f0 = {f0:.1f} Hz\n  Q = {Q:.2f}\n  Gain = {gain_db:.2f} dB")

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.semilogx(freqs, 20 * np.log10(np.abs(H_measured)), label='Measured (Smoothed + Norm)')
plt.semilogx(freqs, 20 * np.log10(np.abs(H_fit)), '--', label='Fitted IIR')
plt.ylabel("Magnitude (dB)")
plt.grid(True)
plt.legend()

plt.subplot(2, 1, 2)
plt.semilogx(freqs, np.angle(H_measured, deg=True), label='Measured Phase')
plt.semilogx(freqs, np.angle(H_fit, deg=True), '--', label='Fitted Phase')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Phase (degrees)")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig("filter_fit.eps", format='eps')
plt.show()